In [1]:
from lxml.html import fromstring
import requests
import pandas as pd
from IPython.display import Markdown

#INICIA FIREBASE
import firebase_admin
from firebase_admin import firestore

cred_obj = firebase_admin.credentials.Certificate('./apprenova-c3758-firebase-adminsdk-99epx-89d70a8235.json')
default_app = firebase_admin.initialize_app(cred_obj, {
    'databaseURL': '/'
})
db = firestore.client()


#LISTA DE CIDADES, NUMERO IDENTIFICADOR E URL DE ACESSO.
def consulta(year, doc, url1turno, url2turno, summ):
    ref = db.collection(year).document(doc)
    
    url = url1turno
    req = requests.get(url)
    root = fromstring(req.text)
    ul = root.xpath('.//ul[@id="main_ul"]')[0]
    cities = []
    for li in ul.xpath('.//li')[0:]:
        cities.append({
            'name': li.xpath('.//a')[0].text_content(),
            'href': li.xpath('.//a')[0].attrib['href'],
            'url': url + li.xpath('.//a')[0].attrib['href']
        })

    #LISTA DE ELEITOS DO ACESSO AO RS 
    url = cities[0]["url"]
    req = requests.get(url)
    root = fromstring(req.text)
    tables = root.xpath(f'.//table[@summary="{summ}"]')
    h2_s = root.xpath('.//h2')
    data = []
    eleitos_class = ["lr_even eleito ", "eleito "]
    id = cities[0]["href"]
    city = cities[0]["name"]
    for index,table in enumerate(tables):
        data.append({
            'city': city,
            'turno': "PRIMEIRO TURNO",
            'title': h2_s[index].text_content(),
            'values': []
        })
        for classe in eleitos_class:
            for tr in table.xpath(f'.//tr[@class="{classe}"]'):
                data[-1]['values'].append({ 
                'name': tr.xpath('.//a')[0].text_content(),
                'partido': tr.xpath('.//td')[3].text_content(),
                'votação': tr.xpath('.//td')[4].text_content()
            })
            


    #ELEITOS SEGUNDO TURNO
    url2 = url2turno
    req = requests.get(url2)
    root = fromstring(req.text)
    tables = root.xpath(f'.//table[@summary="{summ}"]')
    h2_s = root.xpath('.//h2')
    eleitos_class = ["lr_even eleito ", "eleito "]
    for index,table in enumerate(tables):
        data.append({
            'city': city,
            'turno': "SEGUNDO TURNO",
            'title': h2_s[index].text_content(),
            'values': []
        })
        for classe in eleitos_class:
            for tr in table.xpath(f'.//tr[@class="{classe}"]'):
                data[-1]['values'].append({ 
                'name': tr.xpath('.//a')[0].text_content(),
                'partido': tr.xpath('.//td')[3].text_content(),
                'votação': tr.xpath('.//td')[4].text_content()
            })
                
    gov = [d for d in data if (d['title'] == 'Governador')]
    if len(gov) > 1:
        data.remove(gov[0])
    # REGISTRA OS DADOS NO FIREBASE, MERGE TRUE NÃO SUBSTITUI O CONTEÚDO QUE JÁ ESTÁ NA COLEÇÃO
    for d in data:
        if d['title'] == 'Governador':
            ref.set({
                'governador': d
            }, merge=True)
        elif d['title'] == 'Senador':
            ref.set({
                'senador': d
            }, merge=True)
        elif d['title'] == 'Deputado Federal':
            ref.set({
                'deputadoFederal': d
            }, merge=True)
        elif d['title'] == 'Deputado Estadual':
            ref.set({
                'deputadoEstadual': d
            }, merge=True)
    
    return data

consulta('2006', 'KdHuXcU3T8cecJJ8CZk3','https://resultados.tre-rs.jus.br/eleicoes/2006/1turno/','https://resultados.tre-rs.jus.br/eleicoes/2006/2turno/RS.html','Votação dos candidatos na UF')

consulta('2010', '8nhvVoPhro9mr5QwH0Nf','https://resultados.tre-rs.jus.br/eleicoes/2010/1turno/','https://resultados.tre-rs.jus.br/eleicoes/2010/2turno/RS.html','Vota��o dos candidatos na UF')

consulta('2014', 'MylavCoc7DT05DBJonBj','https://resultados.tre-rs.jus.br/eleicoes/2014/1turno/','https://resultados.tre-rs.jus.br/eleicoes/2014/2turno/RS.html','Votação dos candidatos na UF')

consulta('2018', 'GvjvOZooNRnIAiCpgYfU','https://resultados.tre-rs.jus.br/eleicoes/2018/1turno/','https://resultados.tre-rs.jus.br/eleicoes/2018/2turno/RS.html','Votação dos candidatos na UF')


[{'city': 'Rio Grande do Sul',
  'turno': 'PRIMEIRO TURNO',
  'title': 'Presidente',
  'values': [{'name': 'FERNANDO HADDAD',
    'partido': 'PT',
    'votação': '1453291'},
   {'name': 'JAIR MESSIAS BOLSONARO',
    'partido': 'PSL',
    'votação': '3353623'}]},
 {'city': 'Rio Grande do Sul',
  'turno': 'PRIMEIRO TURNO',
  'title': 'Senador',
  'values': [{'name': 'PAULO RENATO PAIM',
    'partido': 'PT',
    'votação': '1875245'},
   {'name': 'LUIS CARLOS HEINZE', 'partido': 'PP', 'votação': '2316365'}]},
 {'city': 'Rio Grande do Sul',
  'turno': 'PRIMEIRO TURNO',
  'title': 'Deputado Federal',
  'values': [{'name': 'ONYX DORNELLES LORENZONI',
    'partido': 'DEM',
    'votação': '183518'},
   {'name': 'PAULO ROBERTO SEVERO PIMENTA',
    'partido': 'PT',
    'votação': '133086'},
   {'name': 'MARLON ARATOR SANTOS DA ROSA',
    'partido': 'PDT',
    'votação': '116483'},
   {'name': 'FERNANDA MELCHIONNA E SILVA',
    'partido': 'PSOL',
    'votação': '114302'},
   {'name': 'HENRIQUE FO